In [43]:
using Revise
using RadiativeTransfer
using RadiativeTransfer.PhaseFunction

# ? Should we make a wrapper around LogNormal?
using Distributions
# using BenchmarkTools

## 
## STEP 1: Create the Aerosol
## 

# Aerosol particle distribution and properties 
μ  = 0.3                # Log mean radius
σ  = 6.82               # Log stddev of radius
r_max = 30.0            # Maximum radius
nquad_radius = 2500     # Number of quadrature points for integrating of size dist.
nᵣ = 1.3                # Real part of refractive index
nᵢ = 0.001                # Imag part of refractive index

size_distribution = LogNormal(log(μ), log(σ))

# Create the aerosol
aero = make_univariate_aerosol(size_distribution, r_max, nquad_radius, nᵣ, nᵢ)


UnivariateAerosol{Float64,Float64}(LogNormal{Float64}(μ=-1.2039728043259361, σ=1.9198594718553708), 30.0, 2500, 1.3, 0.001)

In [44]:
λ = 0.55   # Incident wavelength
polarization_type = Stokes_IQUV()
truncation_type   = δBGE(160, 2.0)

δBGE{Float64}(160, 2.0)

In [45]:
model_NAI2 = make_mie_model(NAI2(), aero, λ, polarization_type, truncation_type)

MieModel{NAI2}
  computation_type: NAI2 NAI2()
  aerosol: UnivariateAerosol{Float64,Float64}
  λ: Float64 0.55
  polarization_type: Stokes_IQUV Stokes_IQUV()
  truncation_type: δBGE{Float64}
  wigner_A: Array{Float64}((1, 1, 1)) [0.0]
  wigner_B: Array{Float64}((1, 1, 1)) [0.0]


In [46]:
aerosol_optics_NAI2 = compute_aerosol_optical_properties(model_NAI2);

┌ Info: Fraction of size distribution cut by max radius: 0.8226655996648291 %
└ @ RadiativeTransfer.PhaseFunction /Users/sanghavi/GDrive/code/github/RadiativeTransfer.jl/src/PhaseFunction/mie_helper_functions.jl:263
Computing PhaseFunctions Siewert NAI-2 style ...100%|███| Time: 0:00:01


In [47]:
# Truncate:
aerosol_optics_trunc = PhaseFunction.truncate_phase(truncation_type, aerosol_optics_NAI2)

AerosolOptics{Float64}
  greek_coefs: GreekCoefs{Float64}
  ω̃: Float64 -1.3759941612017619
  k: Float64 -1647.0816476289463
  derivs: Array{Float64}((1,)) [0.0]


In [42]:
μ, w_μ = gausslegendre(length(aerosol_optics_NAI2.greek_coefs.β));

# Reconstruct phase matrix elements:
f₁₁, f₁₂, f₂₂, f₃₃, f₃₄, f₄₄       = PhaseFunction.reconstruct_phase(aerosol_optics_NAI2.greek_coefs, μ);
tf₁₁, tf₁₂, tf₂₂, tf₃₃, tf₃₄, tf₄₄ = PhaseFunction.reconstruct_phase(aerosol_optics_NAI2.greek_coefs, μ);

([0.16298269148936995, 0.14225498932267744, 0.13036325524129816, 0.12198832940936742, 0.11613930312655452, 0.11170500567480003, 0.10755990632629152, 0.10400508382392869, 0.1004176910450943, 0.09721450416282136  …  120.87653775203351, 164.69741430377897, 222.58446416132048, 332.0523669942334, 495.4598040112732, 847.3409748823537, 1536.8231379214337, 3397.6944691857493, 10330.520536329228, 21642.971237428777], [0.005204146118142419, 0.014602435314904728, 0.013605824036580352, 0.009542007858251307, 0.005111238970920529, 0.0008770224435922724, -0.0024684044242297754, -0.005369873951397063, -0.0075340154668958334, -0.009337148037357275  …  0.32761161618552237, 0.3331321332713996, 0.33374269179775573, 0.314596613452137, 0.28018823717318264, 0.17060617176718754, 0.05213855850372077, -0.4430508026865569, -1.0002066563709744, -0.425251170734261], [0.16298269744575178, 0.14225498471810716, 0.13036325538421356, 0.12198832941396343, 0.11613930243577153, 0.1117050056993869, 0.1075599063190964, 0.10

In [48]:
using Plots

LoadError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


In [ ]:
plot(μ, f₁₁)